
## Compute the associated average outward flux

To have a good quality gradient of the distance map, perform a light smooth over it. Define convolution kernels in each direction and use them recursively. 

In [1]:
from vmtk import vmtkscripts
from vmtk import vtkvmtk
import os
import sys
import numpy as np
import random
import networkx as nx
import vtk
from vtk.numpy_interface import dataset_adapter as dsa
import timeit

imgr = vmtkscripts.vmtkImageReader()
imgr.InputFileName = os.path.join(os.getcwd(), 'test-surf-normal-hole.mha')
imgr.Execute()

mc = vmtkscripts.vmtkMarchingCubes()
mc.Image = imgr.Image
mc.Level = 1000.0
mc.Execute()

In [2]:
surfr = vmtkscripts.vmtkSurfaceReader()
surfr.InputFileName = os.path.join(os.getcwd(), 'realtest.stl')
surfr.Execute()

surf = surfr.Surface

Reading STL surface file.


In [3]:
fedges = vtk.vtkFeatureEdges()
fedges.BoundaryEdgesOn()
fedges.FeatureEdgesOff()
fedges.ManifoldEdgesOff()
fedges.SetInputData(surf)
fedges.Update()

ofedges = fedges.GetOutput()

numEdges = ofedges.GetNumberOfPoints()

In [4]:
if numEdges != 0:
    tempcapper = vmtkscripts.vmtkSurfaceCapper()
    tempcapper.Surface = surf
    tempcapper.Interactive = 0
    tempcapper.Execute()
    
    networkSurface = vtk.vtkPolyData()
    networkSurface.DeepCopy(tempcapper.Surface)
else:
    networkSurface = vtk.vtkPolyData()
    networkSurface.DeepCopy(surf)

In [5]:
numCells = surf.GetNumberOfCells()
cellToDelete = random.randrange(0, numCells-1)

networkSurface.BuildLinks()
networkSurface.DeleteCell(cellToDelete)
networkSurface.RemoveDeletedCells()

net = vmtkscripts.vmtkNetworkExtraction()
net.Surface = networkSurface
net.AdvancementRatio = 1.01
net.Execute()

network = net.Network

Progress: 100%Progress: 100%


In [6]:
convert = vmtkscripts.vmtkCenterlinesToNumpy()
convert.Centerlines = network
convert.Execute()
ad = convert.ArrayDict

wrapping vtkPolyData object
converting cell data: 
Topology
converting points
converting point data: 
Radius
converting cell connectivity list


In [7]:
cellDataTopology = ad['CellData']['Topology']

cellDataTopology

In [8]:
endCellIds = []
startCellIds = []
for cellId, cellTop in enumerate(cellDataTopology):
    if (cellTop[1] == -1) or (cellTop[0] == -1):
        endCellIds.append(cellId)

endCellIds

In [9]:
endPointIds = []
for cellId in endCellIds:  
    endPointIds.append(ad['CellData']['CellPointIds'][cellId][-1])

endPointCoordinates = []
for pointId in endPointIds:
    endPointCoordinates.append(ad['Points'][pointId])

In [10]:
endPointCoordinates = np.array(endPointCoordinates)

In [11]:
numCoordinates = endPointCoordinates.shape[0]
numIters = 8
outlist = []
for i in range(numIters):
    start_time = timeit.default_timer()
    
    sourceSelectId = random.randrange(0, numCoordinates)
    
    sourcePoints = endPointCoordinates[sourceSelectId]
    flatSourcePoints = sourcePoints.flatten(order='C').tolist()
    
    targetPoints = np.delete(endPointCoordinates, sourceSelectId, axis=0)
    flatTargetPoints = targetPoints.flatten(order='C').tolist()
    
    if i == 0:

        cl = vmtkscripts.vmtkCenterlines()
        cl.Surface = networkSurface
        cl.SeedSelectorName = 'pointlist'
        cl.SourcePoints = flatSourcePoints
        cl.TargetPoints = flatTargetPoints
        cl.Execute()
        
        delaunay = vtk.vtkUnstructuredGrid()
        delaunay.DeepCopy(cl.DelaunayTessellation)
    
    else:
        cl = vmtkscripts.vmtkCenterlines()
        cl.Surface = networkSurface
        cl.DelaunayTessellation = delaunay
        cl.SeedSelectorName = 'pointlist'
        cl.SourcePoints = flatSourcePoints
        cl.TargetPoints = flatTargetPoints
        cl.Execute()

    cll = vtk.vtkPolyData()
    cll.DeepCopy(cl.Centerlines)

    numCellsInCl = cll.GetNumberOfCells()
    cellsToDelete = []
    for idx in range(numCellsInCl):
        cell = cll.GetCell(idx)
        numPointsInCell = cell.GetNumberOfPoints()
        if numPointsInCell <= 10:
            print(f'found cell {idx} with {numPointsInCell} to delete')
            cellsToDelete.append(idx)

    if len(cellsToDelete) > 0:
        cll.BuildLinks()
        for cellId in cellsToDelete:
            cll.DeleteCell(cellId)
        cll.RemoveDeletedCells()
        
    outlist.append(cll)
    elapsed = timeit.default_timer() - start_time
    print(f'loop iteration {i} time: {elapsed}')

Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...loop iteration 0 time: 66.43843654100783
Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...loop iteration 1 time: 9.953359909006394
Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...loop iteration 2 time: 9.899388674006332
Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...loop iteration 3 time: 9.905807947012363
Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...loop iteration 4 time: 9.662881346012
Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...loop iteration 5 time: 9.66145081599825
Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...loop iteration 6 time: 9.579545624001184
Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...loop iteratio

In [24]:
new = vtk.vtkAppendPolyData()
for data in outlist:  
    new.AddInputData(data)
new.Update()

combined = new.GetOutput()

In [25]:
# label all non-connected regions in an input surface. 
allRegionConnectivity = vtk.vtkPolyDataConnectivityFilter()
allRegionConnectivity.SetInputData(network)
allRegionConnectivity.SetExtractionModeToAllRegions()
allRegionConnectivity.ColorRegionsOn()
allRegionConnectivity.Update()

connectivity = allRegionConnectivity.GetOutput()

In [43]:
pointLocator = vtk.vtkPointLocator()
pointLocator.SetDataSet(connectivity)
pointLocator.BuildLocator()

numCombinedPoints = combined.GetNumberOfPoints()

pointDataArray = np.zeros(shape=numCombinedPoints)
regionIdArray = connectivity.GetPointData().GetArray('RegionId')
for i in range(numCombinedPoints):
    pointId = pointLocator.FindClosestPoint(combined.GetPoint(i))
    pointDataArray[i] = regionIdArray.GetValue(pointId)

In [69]:
wp = dsa.WrapDataObject(combined)

wp.PointData.append(np.array(pointDataList), 'RegionId')

out = wp.VTKObject

sourcePoints.shape

sourcePoints

targetPoints.shape

targetPoints

In [70]:
cv = vmtkscripts.vmtkCenterlineViewer()
cv.Centerlines = wp.VTKObject
cv.PointDataArrayName = 'RegionId'
cv.Execute()

Quit renderer


In [ ]:
cleaner = vtk.vtkCleanPolyData()

In [ ]:
cleaner.PointMergingOn()
cleaner.ConvertStripsToPolysOff()
cleaner.ConvertPolysToLinesOff()
cleaner.ConvertLinesToPointsOff()
cleaner.SetTolerance(0.005)
cleaner.SetInputData(combined)
cleaner.Update()

In [16]:
rend = vmtkscripts.vmtkRenderer()
rend.Execute()

view = vmtkscripts.vmtkSurfaceViewer()
view.vmtkRenderer = rend.vmtkRenderer
view.Opacity = 0.3
view.Surface = surf
view.Execute()

# view = vmtkscripts.vmtkCenterlineViewer()
# view.vmtkRenderer = rend.vmtkRenderer
# view.Centerlines = outlist[1]
# view.Execute()

view2 = vmtkscripts.vmtkCenterlineViewer()
view2.vmtkRenderer = rend.vmtkRenderer
view2.Centerlines = combined
view2.Execute()

Quit renderer


In [ ]:
write = vmtkscripts.vmtkSurfaceWriter()
write.Surface = new.GetOutput()
write.OutputFileName = os.path.join(os.getcwd(), 'test-new.vtp')
write.Execute()